In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/MyDrive/Sun/ML Shock/Final project/eegClassification

In [3]:
import pandas as pd
import numpy as np
import os
import pyarrow.parquet as pq
from scipy.signal import spectrogram

In [4]:

# path = '/content/drive/MyDrive/Sun/ML Shock/Final project/eegClassification/'
path = '../'

# folder_eeg =  'sample_data/test_eegs/' 
# folder_spec =   'sample_data/test_spectrograms/' 
# train = False 
# df = pd.read_csv(path + 'sample_data/test.csv')

folder_eeg =  'sample_data/train_eegs/'
folder_spec =  'sample_data/train_spectrograms/' 
train =  True #
df = pd.read_csv(path + 'sample_data/train.csv')



In [5]:

files = os.listdir(path + folder_eeg)
df_data = df[df['eeg_id'].isin([int(f.split('.')[0]) for f in files])]

if not train:
    df_data['eeg_label_offset_seconds'] = 0
    df_data['spectrogram_label_offset_seconds'] = 0

In [6]:

def preprocessing(df_data, path, folder_eeg, folder_spec, train):
    freq_eeg = 200 #Hz
    class_votes = ['seizure_vote', 'lpd_vote','gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']  
    # classes = [v.split('_')[0] for v in class_votes]
    label_cols = ['expert_consensus','label_id']
    
    
    df_gr = df_data.groupby(['eeg_id', 'spectrogram_id'])
    info = {}
    for name, group in df_gr:
        if train:
            X = group[['eeg_label_offset_seconds', 'spectrogram_label_offset_seconds']+label_cols+class_votes]
        else:
            X = group[['eeg_id','eeg_label_offset_seconds', 'spectrogram_label_offset_seconds']]
        info[name] = X

    # load parquet files
    data = {}
    for k in info.keys():
        d = {}
        d['eeg'] = pq.read_table(path + folder_eeg +  str(k[0])+'.parquet')
        d['spec'] = pq.read_table(path + folder_spec +  str(k[1])+'.parquet')
        d['info'] = info[k]
        data[k] = d

    spec_types = ['LL', 'RL', 'LP', 'RP']
    # select data from data_spec[0] corresponding to each type
    for k in data.keys():
        spec_data = {}
        cols = list(data[k]['spec'].to_pandas().columns)
        spec_data['spec_freq (Hz)'] = np.array([float(c.split('_')[-1]) for c in cols if c.split('_')[0] == 'LL'])
        spec_data['spec_time (s)'] = data[k]['spec'].to_pandas().filter(regex='time').values.reshape(-1)
        
        for spec_type in spec_types:
            spec_data[spec_type] = data[k]['spec'].to_pandas().filter(regex=spec_type).values

        data[k]['spec_sep'] = spec_data

    for k in data.keys():
        eeg_data = {}
        cols = list(data[k]['eeg'].to_pandas().columns)

        eeg_data['eeg_time (s)'] = np.arange(1, data[k]['eeg'].to_pandas().shape[0]+1)/freq_eeg

        for type in cols:
            eeg_data[type] = data[k]['eeg'].to_pandas()[type].values

        data[k]['eeg_sep'] = eeg_data

    ####
    if train:
        item_id = 'label_id'
    else:
        item_id = 'eeg_id'

    data_preprocessed = {}
    for k in data.keys():

        if train:
            votes = data[k]['info'][class_votes].to_numpy()
            votes_list = [votes[i,:] for i in range(votes.shape[0])]
            consensus = data[k]['info']['expert_consensus'].to_list()

        for kk, offset_eeg, offset_spec, i in zip(data[k]['info'][item_id].to_list(), data[k]['info']['eeg_label_offset_seconds'].to_list(), data[k]['info']['spectrogram_label_offset_seconds'].to_list(), range(len(data[k]['info']['spectrogram_label_offset_seconds'].to_list()))):
            
            d = {}
            
            for kkk in data[k]['spec_sep'].keys():
                if kkk == 'spec_freq (Hz)':
                    d[kkk] = data[k]['spec_sep'][kkk]
                elif kkk == 'spec_time (s)':
                    freq_spec = 1/(data[k]['spec_sep'][kkk][1]-data[k]['spec_sep'][kkk][0])
                    start = int((0+offset_spec)*freq_spec)
                    end = int((600+offset_spec)*freq_spec)-1
                    d[kkk] = data[k]['spec_sep'][kkk][start:end]
                else:
                    start = int((0+offset_spec)*freq_spec)
                    end = int((600+offset_spec)*freq_spec)-1
                    if start >= len(data[k]['spec_sep'][kkk]) or end >= len(data[k]['spec_sep'][kkk]):
                        print('issue spec:', start, end, data[k]['spec_sep'][kkk].shape, data[k]['spec_sep'][kkk][start:end, :].shape)
                    else:
                        d['data_spec_'+kkk] = data[k]['spec_sep'][kkk][start:end, :]

            for kkk in data[k]['eeg_sep'].keys():
                start = int((0+offset_eeg)*freq_eeg)
                end = int((49+offset_eeg)*freq_eeg)
                if start >= len(data[k]['eeg_sep'][kkk]) or end >= len(data[k]['eeg_sep'][kkk]):
                    print('issue eeg:', start, end, data[k]['eeg_sep'][kkk].shape, data[k]['eeg_sep'][kkk][start:end].shape)
                else:
                    if kkk == 'eeg_time (s)':
                        d[kkk] = data[k]['eeg_sep'][kkk][start:end]
                    else:
                        d['data_eeg_'+kkk] = data[k]['eeg_sep'][kkk][start:end]

            d['freq'] = (freq_eeg, freq_spec)
            if train:
                d['class_label'] = consensus[i]
                d['class_votes'] = votes_list[i]
                d['class_probs'] = votes_list[i]/np.sum(votes_list[i])
            d['labels'] = class_votes

            del freq_spec

            data_preprocessed[kk] = d
            
    # first ken in data_processed
    kk = [k for k in data_preprocessed.keys()][0]
    eeg_types = [x for x in data_preprocessed[kk].keys() if 'data_eeg' in x]
    for kk in data_preprocessed.keys():
        for i, k in enumerate(eeg_types): 
            _, _, Sxx = spectrogram(data_preprocessed[kk][k], fs=data_preprocessed[kk]['freq'][0])
            data_preprocessed[kk][k+'_spec'] = Sxx

    return  data_preprocessed

In [7]:
data_preprocessed = preprocessing(df_data, path, folder_eeg, folder_spec, train)

## Check data is consistent

In [8]:
special_keys = ['spec_freq (Hz)', 'spec_time (s)','eeg_time (s)','freq', 
                    'class_label','class_votes','class_probs','labels',]

In [9]:
shapes = []
freqs = []
count = 0
for kk in data_preprocessed.keys():
    for k in data_preprocessed[kk].keys():
        if k not in special_keys:
            if data_preprocessed[kk][k].shape not in shapes:
                shapes.append(data_preprocessed[kk][k].shape)
        if data_preprocessed[kk]['freq'] not in freqs:
            freqs.append(data_preprocessed[kk]['freq'])
        
    count+=1

In [10]:
count, shapes, freqs
# train 
# (69, [(299, 100), (9800,), (129, 43)], [(200, 0.5)])
# test
# (1, [(299, 100), (9800,), (129, 43)], [(200, 0.5)])

(69, [(299, 100), (9800,), (129, 43)], [(200, 0.5)])